In [13]:
import os
import pickle
import pandas as pd
import numpy as np
import time
import json
import pickle
import requests
import tiktoken
import getpass
import logging
from tqdm import tqdm
from tqdm.notebook import tqdm
from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain_community.embeddings import ClovaXEmbeddings
from langchain_community.chat_models import ChatClovaX
from langchain.vectorstores import Milvus
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
# 로깅 설정
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
# 환경설정
load_dotenv(dotenv_path=r"")
api_key = os.getenv("NCP_CLOVASTUDIO_API_KEY")
api_url = os.getenv("NCP_CLOVASTUDIO_API_URL", "https://clovastudio.stream.ntruss.com/")
milvus_host = os.getenv("MILVUS_HOST", "localhost")
milvus_port = os.getenv("MILVUS_PORT", "19530")
os.environ["NCP_CLOVASTUDIO_API_KEY"] = api_key
os.environ["NCP_CLOVASTUDIO_API_URL"] = api_url

try:
    ncp_embeddings = ClovaXEmbeddings(model="bge-m3")
    llm_clova = ChatClovaX(model="HCX-003", max_tokens=2048)
    logger.info("ClovaX Embeddings 및 Chat 모델 초기화 완료")
except Exception as e:
    logger.error(f"ClovaX 모델 초기화 실패: {e}")
    raise

2025-04-03 15:00:40,897 - INFO - ClovaX Embeddings 및 Chat 모델 초기화 완료


In [ ]:
# 단순 텍스트 스플리터
def split_text(text, chunk_size=2000, overlap=250):
    if not text or pd.isnull(text):
        return [""]
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i : i + chunk_size])
    return chunks


# RecursiveCharacterTextSplitter library recursive splitter
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, chunk_overlap=250, separators=["\n\n", "\n", "."]
)

In [ ]:
df = pd.read_csv(
    r""
)

In [ ]:
df["발행일"]

In [ ]:
# 메타데이터 및 벡터 문서 컬럼 설정
metadata_columns = [
    "ISBN",
    "페이지",
    "가격",
    "제목",
    "부제",
    "저자",
    "분류",
    "목차",
    "발행자",
    "표지",
]
vector_doc_columns = [
    "제목",
    "부제",
    "분류",
    "저자",
    "저자소개",
    "책소개",
    "출판사리뷰",
    "추천사",
    "목차",
]

In [ ]:
# 청크 생성
RAG_DB = []
for index, row in df.iterrows():
    metadata = {col: row.get(col, None) for col in metadata_columns}

    # 목차 : 단순 스플리터 적용
    toc_text = row.get("목차", "")
    if pd.isnull(toc_text):
        toc_text = ""
    toc_chunks = split_text(toc_text, chunk_size=2000, overlap=250)

    # non-목차 : 목차 컬럼 제외한 나머지 컬럼은 하나의 텍스트로 연결 후 recursive splitter 적용
    non_toc_text = ""
    for col in vector_doc_columns:
        if col == "목차":
            continue
        value = row.get(col, "")
        if pd.isnull(value):
            value = ""
        non_toc_text += f"{col}: {value}\n"
    recursive_chunks = recursive_splitter.split_text(non_toc_text)

    # 다른 방식의 두 청크들에 동일 Document로 추가
    for chunk in recursive_chunks:
        RAG_DB.append({"text": chunk, "metadata": metadata})
    for chunk in toc_chunks:
        RAG_DB.append({"text": f"목차: {chunk}", "metadata": metadata})

documents = [
    Document(page_content=entry["text"], metadata=entry["metadata"]) for entry in RAG_DB
]

In [ ]:
# 청크 개수
total_chunks = len(documents)
logger.info(f"총 청크 개수: {total_chunks}")

2025-04-04 11:30:27,464 - INFO - 총 청크 개수: 116218


In [ ]:
# 1. HTTP 로그X
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)

In [ ]:
# batch size
batch_size = 5000
total_chunks = len(documents)
batch_dir = r"C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\final_embedding"
os.makedirs(batch_dir, exist_ok=True)

In [ ]:
# 배치 임베딩 파일 불러오기
progress = {}
for batch_idx in range((total_chunks // batch_size) + 1):
    file_path = os.path.join(batch_dir, f"embedding_progress_{batch_idx}.pkl")
    if os.path.exists(file_path):
        with open(file_path, "rb") as f:
            try:
                loaded = pickle.load(f)
                progress.update(loaded)
            except Exception as e:
                print(f"[경고] {file_path} 로드 실패: {e}")

processed_count = len(progress)
remaining_count = total_chunks - processed_count
print(f"[INFO] 이미 처리된 청크: {processed_count}, 남은 청크: {remaining_count}")

[INFO] 이미 처리된 청크: 0, 남은 청크: 116218


In [ ]:
# 임베딩
for i, doc in enumerate(tqdm(documents, desc="임베딩 처리 중", unit="doc")):
    if i in progress:
        continue

    text_chunk = doc.page_content
    embedding = None

    while embedding is None:
        try:
            embedding = ncp_embeddings.embed_query(text_chunk)
        except Exception as e:
            print(f"[에러] 문서 {i} 임베딩 실패: {e}")
            if "429" in str(e):
                print("[경고] 요청 제한 초과. 10초 대기 후 재시도")
                time.sleep(8)
                continue
            embedding = None

    progress[i] = {"text": text_chunk, "embedding": embedding, "metadata": doc.metadata}

    # 배치 단위 저장
    if len(progress) % batch_size == 0:
        batch_idx = len(progress) // batch_size - 1
        temp_path = os.path.join(batch_dir, f"embedding_progress_{batch_idx}.pkl.temp")
        final_path = os.path.join(batch_dir, f"embedding_progress_{batch_idx}.pkl")
        with open(temp_path, "wb") as f:
            pickle.dump(
                {k: v for k, v in progress.items() if k // batch_size == batch_idx}, f
            )
        os.replace(temp_path, final_path)

    time.sleep(0.8)

임베딩 처리 중:   0%|          | 0/116218 [00:00<?, ?doc/s]

[에러] 문서 60 임베딩 실패: Error response 429 while fetching https://clovastudio.stream.ntruss.com/testapp/v1/api-tools/embedding/v2: {"status":{"code":"42901","message":"Too many requests - rate exceeded"},"result":null}
[경고] 요청 제한 초과. 10초 대기 후 재시도
[에러] 문서 120 임베딩 실패: Error response 429 while fetching https://clovastudio.stream.ntruss.com/testapp/v1/api-tools/embedding/v2: {"status":{"code":"42901","message":"Too many requests - rate exceeded"},"result":null}
[경고] 요청 제한 초과. 10초 대기 후 재시도
[에러] 문서 180 임베딩 실패: Error response 429 while fetching https://clovastudio.stream.ntruss.com/testapp/v1/api-tools/embedding/v2: {"status":{"code":"42901","message":"Too many requests - rate exceeded"},"result":null}
[경고] 요청 제한 초과. 10초 대기 후 재시도
[에러] 문서 240 임베딩 실패: Error response 429 while fetching https://clovastudio.stream.ntruss.com/testapp/v1/api-tools/embedding/v2: {"status":{"code":"42901","message":"Too many requests - rate exceeded"},"result":null}
[경고] 요청 제한 초과. 10초 대기 후 재시도
[에러] 문서 300 임베딩 실패: Error res

In [ ]:
# 마지막 배치
last_batch_idx = len(progress) // batch_size
temp_path = os.path.join(batch_dir, f"embedding_progress_{last_batch_idx}.pkl.temp")
final_path = os.path.join(batch_dir, f"embedding_progress_{last_batch_idx}.pkl")
with open(temp_path, "wb") as f:
    pickle.dump(
        {k: v for k, v in progress.items() if k // batch_size == last_batch_idx}, f
    )
os.replace(temp_path, final_path)

print(f"[INFO] 총 처리된 청크: {len(progress)} / {total_chunks}")

[INFO] 총 처리된 청크: 116218 / 116218


In [ ]:
batch_dir = r"C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\final_embedding"
final_output_path = os.path.join(batch_dir, "final_embedding.pkl")

merged_progress = {}

# 병합 0~23 batch
for batch_idx in range(24):
    file_path = os.path.join(batch_dir, f"embedding_progress_{batch_idx}.pkl")
    if os.path.exists(file_path):
        with open(file_path, "rb") as f:
            try:
                data = pickle.load(f)
                merged_progress.update(data)
            except Exception as e:
                print(f"[경고] {file_path} 로드 실패: {e}")
    else:
        print(f"[경고] 파일 없음: {file_path}")

with open(final_output_path, "wb") as f:
    pickle.dump(merged_progress, f)

print(f"[INFO] 병합 완료. 총 청크 수: {len(merged_progress)}")

[INFO] 병합 완료. 총 청크 수: 116218
